In [4]:
import torch
import time
import os
import librosa
import soundfile as sf 
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from optimum.onnxruntime import ORTModelForSpeechSeq2Seq
from datasets import load_dataset 
import numpy as np 

In [5]:
model_id = "openai/whisper-large-v3"
onnx_output_dir = "./whisper-large-v3-onnx" 


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device.upper()}")


if device == "cuda":
    onnx_provider = "CUDAExecutionProvider"
    print("ONNX Runtime будет использовать CUDAExecutionProvider (GPU)")
else:
    onnx_provider = "CPUExecutionProvider"
    print("ONNX Runtime будет использовать CPUExecutionProvider (CPU)")

audio_path = "sample.wav"

n_warmup_runs = 1 
n_timed_runs = 3  

print(f"ID модели: {model_id}")
print(f"Папка для ONNX: {onnx_output_dir}")
print(f"Путь к аудиофайлу: {audio_path}")

Используемое устройство: CUDA
ONNX Runtime будет использовать CUDAExecutionProvider (GPU)
ID модели: openai/whisper-large-v3
Папка для ONNX: ./whisper-large-v3-onnx
Путь к аудиофайлу: sample.wav


In [ ]:

import os
import soundfile as sf 
import numpy as np
import re 

print("\n--- Загрузка данных для WER ВРУЧНУЮ из ЛОКАЛЬНОЙ ПАПКИ ---")

data_base_path = "/home/ubuntu/model-compression-2025/data"


num_samples_for_wer = 15 
target_sampling_rate = 16000 


test_clean_path = os.path.join(data_base_path, "LibriSpeech", "test-clean")
print(f"Поиск данных в директории: {test_clean_path}")

wer_audio_samples = []
wer_reference_texts = []


if not os.path.isdir(test_clean_path):
    print(f"\n!!! ОШИБКА: Директория '{test_clean_path}' не найдена.")
    print(f"!!! Убедитесь, что вы правильно указали 'data_base_path' и папка 'LibriSpeech/test-clean' существует внутри нее.")
else:
    print("Начинаем обход директорий и чтение файлов...")
    samples_collected = 0
    try:
        # Обходим все поддиректории test-clean рекурсивно
        for root, dirs, files in os.walk(test_clean_path):
            # Ищем файл транскрипции в текущей директории
            transcription_file = None
            for file in files:
                if file.endswith(".trans.txt"):
                    transcription_file = os.path.join(root, file)
                    break 

            if transcription_file:

                chapter_transcripts = {}
                try:
                    with open(transcription_file, 'r', encoding='utf-8') as f:
                        for line in f:
                            line = line.strip()
                            # Разделяем по первому пробелу: ID и Текст
                            match = re.match(r'^(\S+)\s+(.*)$', line)
                            if match:
                                file_id = match.group(1)
                                text = match.group(2)
                                chapter_transcripts[file_id] = text.upper() 
                           
                except Exception as e:
                    print(f"Ошибка при чтении файла транскрипции {transcription_file}: {e}")
                    continue 

               
                for file in files:
                    if file.endswith(".flac"):
                        file_id = os.path.splitext(file)[0] # Получаем ID файла без расширения

                        if file_id in chapter_transcripts:
                            audio_file_path = os.path.join(root, file)
                            try:
                                # Читаем аудиофайл
                                audio_data, sr = sf.read(audio_file_path, dtype='float32')

                                # Проверяем частоту дискретизации
                                if sr != target_sampling_rate:
                                    print(f"Предупреждение: Частота дискретизации файла {audio_file_path} ({sr}Hz) отличается от целевой ({target_sampling_rate}Hz). Пропускаем.")
                                   
                                    continue # Пропускаем этот файл, так как он не 16кГц

                               
                                wer_audio_samples.append(audio_data)
                                wer_reference_texts.append(chapter_transcripts[file_id])
                                samples_collected += 1


                                if samples_collected >= num_samples_for_wer:
                                    print(f"\nСобрано {samples_collected} сэмплов.")
                                    raise StopIteration # Используем исключение для выхода из всех циклов

                            except StopIteration: 
                                raise
                            except Exception as e:
                                print(f"Ошибка при чтении или обработке аудиофайла {audio_file_path}: {e}")

    except StopIteration:
        pass
    except Exception as e: 
        print(f"Произошла ошибка во время обхода директорий: {e}")

    # --- Финальная проверка ---
    if samples_collected == num_samples_for_wer:
        print(f"\nУспешно загружено и подготовлено {len(wer_audio_samples)} аудио и {len(wer_reference_texts)} эталонных текстов.")
        if wer_reference_texts:
            print(f"Пример эталонного текста (0): {wer_reference_texts[0]}")
            print(f"Пример формы аудио (0): {wer_audio_samples[0].shape}, Тип: {wer_audio_samples[0].dtype}")
    elif samples_collected > 0:
         print(f"\nПредупреждение: Удалось собрать только {samples_collected} из {num_samples_for_wer} запрошенных сэмплов.")
    else:
        print("\n!!! Не удалось собрать ни одного сэмпла. Проверьте структуру папок и наличие файлов в {test_clean_path}.")


# Если списки остались пустыми (из-за ошибок выше), присваиваем пустые списки
if not wer_audio_samples:
    wer_audio_samples = []
if not wer_reference_texts:
    wer_reference_texts = []


--- Загрузка данных для WER ВРУЧНУЮ из ЛОКАЛЬНОЙ ПАПКИ ---
Поиск данных в директории: /home/ubuntu/model-compression-2025/data/LibriSpeech/test-clean
Начинаем обход директорий и чтение файлов...

Собрано 15 сэмплов.

Успешно загружено и подготовлено 15 аудио и 15 эталонных текстов.
Пример эталонного текста (0): WHAT THE LATTER DAY SAINTS CALL CELESTIAL MARRIAGE IS CHARACTERISTIC OF THE CHURCH AND IS IN VERY GENERAL PRACTISE BUT OF CELESTIAL MARRIAGE PLURALITY OF WIVES WAS AN INCIDENT NEVER AN ESSENTIAL
Пример формы аудио (0): (203999,), Тип: float32


In [7]:
print(f"\n--- Загрузка оригинальной модели {model_id} (PyTorch) ---")
start_load_torch = time.time()

original_processor = WhisperProcessor.from_pretrained(model_id)
original_model = WhisperForConditionalGeneration.from_pretrained(model_id)

original_model.to(device)
original_model.eval()

end_load_torch = time.time()
print(f"Оригинальная модель и процессор загружены за {end_load_torch - start_load_torch:.2f} сек.")
print(f"Модель PyTorch размещена на: {next(original_model.parameters()).device}")


--- Загрузка оригинальной модели openai/whisper-large-v3 (PyTorch) ---


Оригинальная модель и процессор загружены за 5.33 сек.
Модель PyTorch размещена на: cuda:0


In [5]:
print(f"\n--- Экспорт модели в ONNX (в папку {onnx_output_dir}) ---")


encoder_path = os.path.join(onnx_output_dir, "encoder_model.onnx")
decoder_path = os.path.join(onnx_output_dir, "decoder_model.onnx")

if not (os.path.exists(encoder_path) and os.path.exists(decoder_path)):
    print("ONNX модель не найдена или неполная, запускаю экспорт...")
    start_export_time = time.time()


    model_for_export = ORTModelForSpeechSeq2Seq.from_pretrained(model_id, export=True)

    model_for_export.save_pretrained(onnx_output_dir)


    original_processor.save_pretrained(onnx_output_dir)

    end_export_time = time.time()
    print(f"Экспорт завершен за {end_export_time - start_export_time:.2f} секунд.")
else:
    print("Найдены существующие ONNX файлы в папке, пропускаю экспорт.")
    if not os.path.exists(os.path.join(onnx_output_dir, "preprocessor_config.json")):
         original_processor.save_pretrained(onnx_output_dir)
         print("Скопирован файл конфигурации процессора в папку ONNX.")

print(f"Файлы ONNX модели должны находиться в: {os.path.abspath(onnx_output_dir)}")


--- Экспорт модели в ONNX (в папку ./whisper-large-v3-onnx) ---
ONNX модель не найдена или неполная, запускаю экспорт...
ВНИМАНИЕ: Экспорт может занять ОЧЕНЬ много времени (десятки минут и более) и потребовать много RAM!


Moving the following attributes in the config to the generation config: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
/home/ubuntu/miniconda3/envs/model-compression/lib/python3.12/site-packages/transformers/models/whisper/modeling_whisper.py:1016: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_features.shape[-1] != expected_seq_length:
/home/ubuntu/miniconda3/envs/model-compression/lib/python3.12/site-packages/transformers/models/whisper/modeling_whisper.py:334: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will

Экспорт завершен за 125.85 секунд.
Файлы ONNX модели должны находиться в: /home/ubuntu/model-compression-2025/whisper-large-v3-onnx


In [8]:
print(f"\n--- Загрузка ONNX модели из {onnx_output_dir} ---")
start_load_onnx = time.time()

onnx_processor = WhisperProcessor.from_pretrained(onnx_output_dir)

onnx_model = ORTModelForSpeechSeq2Seq.from_pretrained(onnx_output_dir, provider=onnx_provider)

end_load_onnx = time.time()
print(f"ONNX модель и процессор загружены за {end_load_onnx - start_load_onnx:.2f} сек.")


--- Загрузка ONNX модели из ./whisper-large-v3-onnx ---


2025-05-12 22:45:55.495893053 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-12 22:45:55.495954926 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
2025-05-12 22:45:56.753307239 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-12 22:45:56.753352621 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


ONNX модель и процессор загружены за 4.89 сек.


2025-05-12 22:45:58.351383530 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-12 22:45:58.351429273 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [ ]:

import psutil # Для замера RAM
import torch.cuda # Для замера VRAM

print(f"\n--- Запуск инференса (PyTorch на {device}) для {len(wer_audio_samples)} сэмплов ---")

torch_total_time = 0.0
torch_predictions = []
process = psutil.Process(os.getpid()) # Получаем текущий процесс Python

if device == "cuda":
    torch.cuda.reset_peak_memory_stats(device)
    torch.cuda.empty_cache() # Попытка очистить кэш перед замером

ram_before_torch = process.memory_info().rss # RAM до начала цикла

if wer_audio_samples:
    print("Прогрев PyTorch модели...")
    try:
        with torch.no_grad():

             inputs_warmup = original_processor(wer_audio_samples[0], sampling_rate=16000, return_tensors="pt")
             input_features_warmup = inputs_warmup.input_features.to(device)
             _ = original_model.generate(input_features_warmup) 
             if device == "cuda":
                 torch.cuda.synchronize()
        print("Прогрев завершен.")
    except Exception as e:
        print(f"Ошибка во время прогрева PyTorch: {e}")


print(f"Запуск инференса и замера времени для {len(wer_audio_samples)} сэмплов...")
start_full_torch_time = time.time()
with torch.no_grad():
    for i, audio_sample in enumerate(wer_audio_samples):

        inputs = original_processor(audio_sample, sampling_rate=16000, return_tensors="pt")

        input_features_torch = inputs.input_features.to(device)

        iter_start_time = time.time()
        
        predicted_ids = original_model.generate(input_features_torch)
        if device == "cuda":
            torch.cuda.synchronize()
        iter_end_time = time.time()

        run_time = iter_end_time - iter_start_time
        torch_total_time += run_time

        predicted_ids_cpu = predicted_ids.to("cpu")
        transcription = original_processor.batch_decode(predicted_ids_cpu, skip_special_tokens=True)[0]
        torch_predictions.append(transcription.upper()) # Сохраняем в верхнем регистре


ram_after_torch = process.memory_info().rss
vram_peak_torch = torch.cuda.max_memory_allocated(device) if device == "cuda" else 0
end_full_torch_time = time.time()
print(f"Инференс PyTorch завершен за {end_full_torch_time - start_full_torch_time:.2f} сек (общее время цикла).")



avg_torch_time_per_sample = torch_total_time / len(wer_audio_samples) if wer_audio_samples else 0
print(f"\nСреднее время инференса на сэмпл (PyTorch): {avg_torch_time_per_sample:.4f} сек")


torch_ram_usage = ram_after_torch - ram_before_torch
print(f"Приблизительное изменение RAM во время инференса PyTorch: {torch_ram_usage / (1024**2):.2f} MB")
if device == "cuda":
    print(f"Пиковое потребление VRAM (отслеженное PyTorch): {vram_peak_torch / (1024**2):.2f} MB")

if torch_predictions:
    print(f"\nПример предсказания (PyTorch, 0): {torch_predictions[0]}")

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.



--- Запуск инференса (PyTorch на cuda) для 15 сэмплов ---
Прогрев PyTorch модели...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Прогрев завершен.
Запуск инференса и замера времени для 15 сэмплов...
Инференс PyTorch завершен за 14.99 сек (общее время цикла).

Среднее время инференса на сэмпл (PyTorch): 0.9807 сек
Приблизительное изменение RAM во время инференса PyTorch: 265.25 MB
Пиковое потребление VRAM (отслеженное PyTorch): 6536.38 MB

Пример предсказания (PyTorch, 0):  WHAT THE LATTER-DAY SAINTS CALL CELESTIAL MARRIAGE IS CHARACTERISTIC OF THE CHURCH AND IS IN VERY GENERAL PRACTICE BUT OF CELESTIAL MARRIAGE PLURALITY OF WIVES WAS AN INCIDENT NEVER AN ESSENTIAL


In [11]:

import psutil # Для замера RAM
import torch.cuda # Для замера VRAM

print(f"\n--- Запуск инференса (ONNX на {onnx_provider}) для {len(wer_audio_samples)} сэмплов ---")

onnx_total_time = 0.0
onnx_predictions = []
process = psutil.Process(os.getpid()) # Обновляем ссылку на процесс


if device == "cuda":
    torch.cuda.reset_peak_memory_stats(device)
    torch.cuda.empty_cache() # Попытка очистить кэш

ram_before_onnx = process.memory_info().rss # RAM до начала цикла

# Прогрев (опционально, на первом сэмпле)
if wer_audio_samples:
    print("Прогрев ONNX модели...")
    try:

        inputs_warmup = onnx_processor(wer_audio_samples[0], sampling_rate=16000, return_tensors="pt")

        input_features_warmup_onnx = inputs_warmup.input_features.to(device)
        _ = onnx_model.generate(input_features_warmup_onnx)
        
        if device == "cuda":
             torch.cuda.synchronize()
        print("Прогрев завершен.")
    except Exception as e:
        print(f"Ошибка во время прогрева ONNX: {e}")


print(f"Запуск инференса и замера времени для {len(wer_audio_samples)} сэмплов...")
start_full_onnx_time = time.time()
for i, audio_sample in enumerate(wer_audio_samples):

    inputs = onnx_processor(audio_sample, sampling_rate=16000, return_tensors="pt")

    input_features_for_onnx_generate = inputs.input_features.to(device)

    iter_start_time = time.time()

    predicted_ids = onnx_model.generate(input_features_for_onnx_generate)

    if device == "cuda":
        torch.cuda.synchronize()
    iter_end_time = time.time()

    run_time = iter_end_time - iter_start_time
    onnx_total_time += run_time

    predicted_ids_cpu = predicted_ids.to("cpu") 
    transcription = onnx_processor.batch_decode(predicted_ids_cpu, skip_special_tokens=True)[0]
    onnx_predictions.append(transcription.upper())


ram_after_onnx = process.memory_info().rss
vram_peak_onnx = torch.cuda.max_memory_allocated(device) if device == "cuda" else 0
end_full_onnx_time = time.time()
print(f"Инференс ONNX завершен за {end_full_onnx_time - start_full_onnx_time:.2f} сек (общее время цикла).")

avg_onnx_time_per_sample = onnx_total_time / len(wer_audio_samples) if wer_audio_samples else 0
print(f"\nСреднее время инференса на сэмпл (ONNX): {avg_onnx_time_per_sample:.4f} сек")

onnx_ram_usage = ram_after_onnx - ram_before_onnx
print(f"Приблизительное изменение RAM во время инференса ONNX: {onnx_ram_usage / (1024**2):.2f} MB")
if device == "cuda":
    print(f"Пиковое потребление VRAM (отслеженное PyTorch*): {vram_peak_onnx / (1024**2):.2f} MB")
    print(f"*Примечание: Замер VRAM для ONNX через torch.cuda может быть неполным, т.к. ONNX RT может управлять памятью GPU самостоятельно.")

if onnx_predictions:
    print(f"\nПример предсказания (ONNX, 0): {onnx_predictions[0]}")


--- Запуск инференса (ONNX на CUDAExecutionProvider) для 15 сэмплов ---
Прогрев ONNX модели...
Прогрев завершен.
Запуск инференса и замера времени для 15 сэмплов...
Инференс ONNX завершен за 11.81 сек (общее время цикла).

Среднее время инференса на сэмпл (ONNX): 0.7684 сек
Приблизительное изменение RAM во время инференса ONNX: 113.38 MB
Пиковое потребление VRAM (отслеженное PyTorch*): 6562.14 MB
*Примечание: Замер VRAM для ONNX через torch.cuda может быть неполным, т.к. ONNX RT может управлять памятью GPU самостоятельно.

Пример предсказания (ONNX, 0):  WHAT THE LATTER-DAY SAINTS CALL CELESTIAL MARRIAGE IS CHARACTERISTIC OF THE CHURCH AND IS IN VERY GENERAL PRACTICE BUT OF CELESTIAL MARRIAGE PLURALITY OF WIVES WAS AN INCIDENT NEVER AN ESSENTIAL


In [ ]:
import jiwer 
import os

print("\n\n--- Сравнительный анализ (Автоматизированный) ---")

# --- 1. Скорость инференса (средняя на сэмпл) ---
print("\n1. Скорость инференса (средняя на сэмпл):")
# Используем переменные из обновленных Ячеек 8 и 9
print(f"   Среднее время (PyTorch): {avg_torch_time_per_sample:.4f} сек")
print(f"   Среднее время (ONNX):    {avg_onnx_time_per_sample:.4f} сек")

if avg_torch_time_per_sample > 0 and avg_onnx_time_per_sample > 0:
    speedup = avg_torch_time_per_sample / avg_onnx_time_per_sample
    improvement = ((avg_torch_time_per_sample - avg_onnx_time_per_sample) / avg_torch_time_per_sample) * 100
    print(f"\n   Ускорение ONNX по сравнению с PyTorch: {speedup:.2f}x")
    print(f"   ONNX быстрее PyTorch на: {improvement:.2f}%")
elif avg_onnx_time_per_sample == 0:
     print("\n   Не удалось рассчитать ускорение (время ONNX равно нулю).")
else:
    print("\n   Не удалось рассчитать ускорение.")


# --- 2. Размер модели ---
print("\n2. Размер модели:")

def get_folder_size_mb(folder_path):
    total_size = 0
    try:
        for dirpath, dirnames, filenames in os.walk(folder_path):
            for f in filenames:
                fp = os.path.join(dirpath, f)
                if not os.path.islink(fp):
                    total_size += os.path.getsize(fp)
    except FileNotFoundError:
        return 0
    return total_size / (1024 * 1024) # в МБ

# Переменная onnx_output_dir должна быть определена в Ячейке 2
onnx_model_size_mb = get_folder_size_mb(onnx_output_dir)
print(f"   Размер папки с ONNX моделью: {onnx_model_size_mb:.2f} МБ")
print("   (Размер оригинальной PyTorch модели large-v3 обычно ~3-6 ГБ, в зависимости от точности)")
print("   Примечание: Значительное уменьшение размера достигается квантованием.")


# --- 3. Потребление памяти (RAM/VRAM) ---
print("\n3. Потребление памяти (Приблизительные замеры):")
# Используем переменные из обновленных Ячеек 8 и 9
print(f"   Изменение RAM (PyTorch): ~ {torch_ram_usage / (1024**2):.2f} MB")
print(f"   Изменение RAM (ONNX):    ~ {onnx_ram_usage / (1024**2):.2f} MB")
print("   Примечание RAM: Замер показывает разницу RSS процесса до и после цикла инференса, может включать сторонние факторы.")

if device == "cuda":
    # Используем переменные из обновленных Ячеек 8 и 9
    print(f"\n   Пик VRAM (PyTorch, отслечено): {vram_peak_torch / (1024**2):.2f} MB")
    print(f"   Пик VRAM (ONNX, отслечено*): {vram_peak_onnx / (1024**2):.2f} MB")
    print(f"   *Примечание VRAM: Замер для ONNX через torch.cuda может быть неполным.")
else:
    print("\n   VRAM: Не измерялось (выполнение на CPU).")


# --- 4. Качество (Word Error Rate - WER) ---
print("\n4. Качество (Word Error Rate):")

wer_calculated = False
# Проверяем наличие всех необходимых списков из Ячеек 3, 8, 9
if 'wer_reference_texts' in globals() and 'torch_predictions' in globals() and 'onnx_predictions' in globals() and \
   wer_reference_texts and torch_predictions and onnx_predictions and \
   (len(wer_reference_texts) == len(torch_predictions) == len(onnx_predictions)):
    try:
        # Расчет WER для PyTorch
        wer_torch_info = jiwer.compute_measures(wer_reference_texts, torch_predictions)
        wer_torch = wer_torch_info['wer'] * 100 # В процентах

        # Расчет WER для ONNX
        wer_onnx_info = jiwer.compute_measures(wer_reference_texts, onnx_predictions)
        wer_onnx = wer_onnx_info['wer'] * 100 # В процентах

        print(f"   WER (PyTorch): {wer_torch:.2f}%")
        print(f"   WER (ONNX):    {wer_onnx:.2f}%")
        print(f"   (Чем ниже WER, тем лучше)")

        # Сравнение
        wer_diff = abs(wer_torch - wer_onnx)
        print(f"   Разница в WER: {wer_diff:.2f}%")
        if wer_diff < 0.1:
            print("   Качество распознавания практически идентично.")
        else:
            print("   Есть заметная разница в качестве распознавания.")

        # Вывод примера для визуальной проверки (первый сэмпл)
        print("\n   Пример сравнения (сэмпл 0):")
        print(f"   Эталон:  \"{wer_reference_texts[0]}\"")
        print(f"   PyTorch: \"{torch_predictions[0]}\"")
        print(f"   ONNX:    \"{onnx_predictions[0]}\"")
        wer_calculated = True

    except Exception as e:
        print(f"\n   Ошибка при расчете WER: {e}")
        print(f"   Убедитесь, что библиотека jiwer установлена: pip install jiwer")

# Диагностика, если WER не рассчитан
if not wer_calculated:
    if 'wer_reference_texts' not in globals() or not wer_reference_texts:
         print("\n   Не удалось рассчитать WER: Нет эталонных текстов (ошибка в Ячейке 3?).")
    elif 'torch_predictions' not in globals() or not torch_predictions:
         print("\n   Не удалось рассчитать WER: Отсутствуют предсказания PyTorch (ошибка в Ячейке 8?).")
    elif 'onnx_predictions' not in globals() or not onnx_predictions:
         print("\n   Не удалось рассчитать WER: Отсутствуют предсказания ONNX (ошибка в Ячейке 9?).")
    elif not (len(wer_reference_texts) == len(torch_predictions) == len(onnx_predictions)):
        print("\n   Не удалось рассчитать WER: Не совпадает количество эталонов и предсказаний.")
        print(f"     Эталонов: {len(wer_reference_texts) if 'wer_reference_texts' in globals() else 'N/A'}")
        print(f"     PyTorch предсказаний: {len(torch_predictions) if 'torch_predictions' in globals() else 'N/A'}")
        print(f"     ONNX предсказаний: {len(onnx_predictions) if 'onnx_predictions' in globals() else 'N/A'}")
    else:
         print("\n   Не удалось рассчитать WER по неизвестной причине.")


print("\n--- Анализ завершен ---")



--- Сравнительный анализ (Автоматизированный) ---

1. Скорость инференса (средняя на сэмпл):
   Среднее время (PyTorch): 0.9807 сек
   Среднее время (ONNX):    0.7684 сек

   Ускорение ONNX по сравнению с PyTorch: 1.28x
   ONNX быстрее PyTorch на: 21.65%

2. Размер модели:
   Размер папки с ONNX моделью: 9460.61 МБ
   (Размер оригинальной PyTorch модели large-v3 обычно ~3-6 ГБ, в зависимости от точности)
   Примечание: Значительное уменьшение размера достигается квантованием.

3. Потребление памяти (Приблизительные замеры):
   Изменение RAM (PyTorch): ~ 265.25 MB
   Изменение RAM (ONNX):    ~ 113.38 MB
   Примечание RAM: Замер показывает разницу RSS процесса до и после цикла инференса, может включать сторонние факторы.

   Пик VRAM (PyTorch, отслечено): 6536.38 MB
   Пик VRAM (ONNX, отслечено*): 6562.14 MB
   *Примечание VRAM: Замер для ONNX через torch.cuda может быть неполным.

4. Качество (Word Error Rate):
   WER (PyTorch): 6.73%
   WER (ONNX):    6.73%
   (Чем ниже WER, тем лучш

In [13]:

import os
import time

# --- Функция для измерения размера папки (если ее нет выше) ---
def get_folder_size_mb(folder_path):
    total_size = 0
    try:
        for dirpath, dirnames, filenames in os.walk(folder_path):
            for f in filenames:
                fp = os.path.join(dirpath, f)
                # Проверяем, что это не символическая ссылка, чтобы избежать двойного подсчета
                if not os.path.islink(fp):
                    total_size += os.path.getsize(fp)
    except FileNotFoundError:
        print(f"Предупреждение: Папка {folder_path} не найдена.")
        return 0
    return total_size / (1024 * 1024) # в МБ


if 'original_model' in globals():
    temp_save_dir = "./whisper-large-v3-original-pytorch" # Имя временной папки
    print(f"\nСохранение оригинальной PyTorch модели в папку: {temp_save_dir}")
    print("Это может занять некоторое время...")
    start_save_time = time.time()


    if os.path.exists(temp_save_dir):
        import shutil
        print(f"Удаление существующей папки {temp_save_dir}...")
        shutil.rmtree(temp_save_dir)


    original_model.save_pretrained(temp_save_dir)


    end_save_time = time.time()
    print(f"Модель сохранена за {end_save_time - start_save_time:.2f} секунд.")

    pytorch_model_size_mb = get_folder_size_mb(temp_save_dir)
    print(f"\nТочный размер сохраненной оригинальной PyTorch модели: {pytorch_model_size_mb:.2f} МБ")

else:
    print("\nОшибка: Переменная 'original_model' не найдена. Убедитесь, что Ячейка 4 была выполнена.")
    pytorch_model_size_mb = 0 


Сохранение оригинальной PyTorch модели в папку: ./whisper-large-v3-original-pytorch
Это может занять некоторое время...


/home/ubuntu/miniconda3/envs/model-compression/lib/python3.12/site-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Модель сохранена за 12.51 секунд.

Точный размер сохраненной оригинальной PyTorch модели: 5888.20 МБ
